In [1]:
from load_data import df
from distance_matrix import add_directions_columns
import pandas as pd

/usr/local/Caskroom/miniconda/base/envs/property-data/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [18]:
HIDE_USE_CODES = [
    "101",
    "132",
    "130",
    "104",
    "111",
    "105",
    "112",
    "109",
    "900",
    "931",
    "934"
]
HIDE_STYLE = [
    "GROUP HOME",
    "APTMNT-CON",
    "THREE FAM",
    "COLONIAL",
    "VICTORIAN",
    "DUPLEX 2S",
    "TWO FAMILY"
]

In [19]:
filtered = df.assign(
    distance=df.geom.distance(list(df[df.SITE_ADDR == "332 MAIN ST"].geom)[0].centroid)
).query(
    'CITY.str.lower() == "greenfield" and USE_CODE != @HIDE_USE_CODES and BLD_AREA > 5000 and distance < 700 and STYLE != @HIDE_STYLE'
)

In [20]:
filtered.explore(   
    column="FULL_STR",
    # tiles="USGS.USImagery",
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='ESRI',
    popup=True,
    style_kwds={"fillOpacity": 0.2}
).save(
    'wharehouse.html'
)

In [21]:
with pd.option_context('display.max_rows', None):
    print(filtered.agg('{0[USE_CODE]}: {0[USE_DESC]}'.format, axis=1).value_counts())

325: Small Retail and Services stores (under 10,000 sq. ft.)                                                 27
031: Mixed Use (Primarily Commercial, some Residential)                                                      18
340: General Office Buildings                                                                                17
013: Mixed Use (Primarily Residential, some Commercial)                                                      12
316: Other Storage, Warehouse, and Distribution facilities (see also usecode 401)                             8
960: Church, Mosque, Synagogue, Temple, etc...                                                                7
400: Buildings for manufacturing operations                                                                   5
957: Charitable Services                                                                                      4
341: Bank Buildings                                                                                     

In [22]:
with pd.option_context('display.max_rows', None):
    print(filtered.STYLE.value_counts())

COMM BLOCK    37
STORE         17
OFFICE        12
MIXED USE      9
CHURCH/SYN     7
WAREHOUSE      7
MILL           5
GOVT BLDG      5
LODGE          4
BANK           4
FUNERAL HM     3
               2
REPAIR GAR     2
RESTAURANT     2
OFFICE-PRO     2
INDUST-LT      1
GYMNASIUM      1
R/M SHOP       1
BAR            1
AUTO DEALR     1
APRTMNT-GN     1
CAMP, YRND     1
CONDO-OFC      1
DEPT STORE     1
SHOP CTR       1
INDUSTRIAL     1
RELAY BLDG     1
Name: STYLE, dtype: int64


In [24]:
filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 130 entries, 2281 to 4387
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   BLDG_VAL    130 non-null    int64   
 1   LAND_VAL    130 non-null    int64   
 2   OTHER_VAL   130 non-null    int64   
 3   TOTAL_VAL   130 non-null    int64   
 4   FY          130 non-null    int64   
 5   LOT_SIZE    130 non-null    float64 
 6   LS_DATE     130 non-null    object  
 7   LS_PRICE    130 non-null    int64   
 8   USE_CODE    130 non-null    object  
 9   SITE_ADDR   130 non-null    object  
 10  ADDR_NUM    130 non-null    object  
 11  FULL_STR    130 non-null    object  
 12  LOCATION    130 non-null    object  
 13  CITY        130 non-null    object  
 14  ZIP         130 non-null    object  
 15  OWNER1      130 non-null    object  
 16  OWN_ADDR    130 non-null    object  
 17  OWN_CITY    130 non-null    object  
 18  OWN_STATE   130 non-null    object  
 

In [25]:
filtered.to_csv('wharehouse.csv')